In [1]:
print("oK")

oK


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
PINECONE_API_KEY="ef6d21a0-5c08-4de8-aab8-e28778d54cb4"
# PINECONE_API_ENV=""

In [ ]:
#Extract Data from PDF
def load_pdf(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documets=loader.load()
    return documets
